In [1]:
import TIdatabase as ti
%matplotlib inline 
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import statsmodels.api as sm
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from matplotlib import rcParams
%reload_ext rpy2.ipython
from rpy2.robjects import pandas2ri

In [4]:
%%R
# dftouse = read.csv("collegedata_normalized.csv",header=T)
head(dftouse)

  X  studentID classrank admissionstest AP   averageAP  SATsubject          GPA
1 0 PWY05BUB4I        NA     0.11101293  7  0.18742697  0.07064293 -0.013895163
2 1 3UVDFVI9Z0        NA     0.03509851  7  0.11599840 -0.09602374  0.036646353
3 2 BCCBHJUP0M        NA     0.03509851  0          NA  0.07064293  0.029426136
4 3 WZFPWHSQMS        NA     0.16622342  7  0.15171269  0.23730959  0.007765486
5 4 5W1JNQA7G0        NA     0.04890113  1 -0.06257303 -0.09602374  0.040256461
6 5 TWUKL79B6V        NA     0.04890113  0          NA -0.09602374  0.058307003
         GPA_w                program schooltype intendedgradyear
1  0.005108614 Biomedical engineering         -1             2017
2  0.033997503               Classics         -1             2013
3 -0.088224719     Biological Science          1             2016
4 -0.032669164                Physics          1             2015
5           NA                                 1             2011
6           NA      Political Science       